In [1]:
import librosa
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from datasets import load_dataset, Audio, DatasetDict, ClassLabel
from torch.utils.data import DataLoader
import tqdm

In [2]:
def adjust_labels(batch):
    batch["emotion"] = [sentiment for sentiment in batch["emotion"]]
    return batch

In [3]:
english_dataset = load_dataset("./dataset/ravd", data_dir="./", split="train")
features = english_dataset.features.copy()
features["emotion"] = ClassLabel(names=['happy','neutral','angry','sad','fearful','disgust','calm','surprised'])
english_dataset = english_dataset.map(adjust_labels, batched=True, features=features)
english_dataset = english_dataset.train_test_split(test_size=0.2,stratify_by_column="emotion")
test_data_split = english_dataset["test"].train_test_split(test_size=0.5,stratify_by_column="emotion")
english_dataset = DatasetDict({
    "train": english_dataset["train"],
    "test": test_data_split["test"],
    "val": test_data_split["train"]
})



Resolving data files:   0%|          | 0/1441 [00:00<?, ?it/s]

In [4]:
def feature_extraction(examples):
    audio_arrays = [[x["array"], x["sampling_rate"]] for x in examples["audio"]]
    # extract the features from the audio
    inputs = []
    for audio in audio_arrays:
        # Assuming audio[0] is your audio array and audio[1] is the sampling rate
        audio_duration = len(audio[0]) / audio[1]  # Calculate the duration of the audio in seconds
        
        # Set the maximum duration you desire
        max_duration = 3  # For example, 3 seconds
        
        # If the audio duration exceeds the maximum duration, trim it
        if audio_duration >= max_duration:
            audio[0] = audio[0][:int(max_duration * audio[1])]
        
        else:
            samples_to_pad = int((max_duration - audio_duration) * audio[1])
            
            # Pad the audio with zeros
            padded_audio = np.pad(audio[0], (0, samples_to_pad), 'constant')
            audio[0] = padded_audio

        mfcc = librosa.feature.mfcc(y=audio[0], sr=16000, n_mfcc=128)
        inputs.append([mfcc])
    return {'mfcc': inputs}



In [5]:
english_dataset = english_dataset.map(feature_extraction, remove_columns="audio", batched=True)

english_dataset

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['emotion', 'mfcc'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['emotion', 'mfcc'],
        num_rows: 144
    })
    val: Dataset({
        features: ['emotion', 'mfcc'],
        num_rows: 144
    })
})

In [6]:
english_dataset = english_dataset.rename_column("emotion", "label")

In [7]:
class CNN(nn.Module):

    def __init__(self, no_features,no_labels):
        super().__init__()
        self.cnn_stack = nn.Sequential(
            # first layer
            nn.BatchNorm2d(no_features),
            nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(),
            # second layer
            
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(),

            # third layer
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2,  stride=2),
            nn.Dropout(),

            # fourth layer
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=(3,5),  stride=(3,5)),
            nn.Dropout(),

            # fifth layer
            nn.Conv2d(in_channels=128,out_channels=64,kernel_size=3,stride=1,padding=2),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=(3,5),  stride=(3,5)),
            nn.Dropout(),
            
           
            # fully connected layer
            nn.Flatten(),
            nn.Linear(in_features=128,out_features=256),
            nn.ELU(),
            nn.Dropout(),            

            # output layer
            nn.Linear(in_features=256,out_features=no_labels)
        )

    
    def forward(self, x):
        return self.cnn_stack(x)


    def predict(self, x, labels, device):
        x = x.to(device)
        labels = labels.to(device)
        preds = self.forward(x)
        total = labels.shape[0]
        correct = 0
        _, predicted = torch.max(preds, 1)
        correct += (predicted == labels).sum().item()
        return predicted , correct / total
        



In [8]:

def train_single_epoch(model,dataloader,loss_fn,optimizer,device):
    for batch in dataloader:
        mfcc = batch['mfcc'].to(device)
        label = batch['label'].to(device)
        logits=model(mfcc)
        loss=loss_fn(logits.float(),label.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        _, acc = model.predict(val["mfcc"], val["label"], device)
    print(f"loss:{loss.item()}", "val accuracy:", acc)
    
def train(model,dataloader,loss_fn,optimizer,device,epochs):
    for i in tqdm.tqdm(range(epochs)):
        print(f"epoch:{i+1}")
        train_single_epoch(model,dataloader,loss_fn,optimizer,device)
        print('-------------------------------------------')
    print('Finished Training')

In [9]:
dataloader = DataLoader(english_dataset["train"].with_format("torch"), batch_size=32)

val = english_dataset["val"].with_format('torch')
device = torch.device('cuda:0')
model = CNN(1,8)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
torch.cuda.is_available()
train(model,dataloader,loss_fn,optimizer,device,200)

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

epoch:1


  0%|▍                                                                                 | 1/200 [00:13<45:08, 13.61s/it]

loss:2.193697929382324 val accuracy: 0.13194444444444445
-------------------------------------------
epoch:2


  1%|▊                                                                                 | 2/200 [00:24<38:56, 11.80s/it]

...

loss:0.5892530083656311 val accuracy: 0.5416666666666666
-------------------------------------------
epoch:199


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [31:39<00:09,  9.26s/it]

loss:0.44052186608314514 val accuracy: 0.4930555555555556
-------------------------------------------
epoch:200


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [31:51<00:00,  9.56s/it]

loss:0.476376473903656 val accuracy: 0.5069444444444444
-------------------------------------------
Finished Training


In [10]:
test = english_dataset["test"].with_format("torch")

In [12]:
preds = model(test["mfcc"].to(device))
labels = test["label"].to(device)

In [13]:
print(preds)

tensor([[ -0.6741,  -8.3026,   5.8219,  ...,   0.0358, -11.6334,   5.8354],
        [  6.9167,  -3.8829,  -1.4376,  ...,  -8.5275,  -7.7291,  -2.3084],
        [ -4.9443,   0.4118,  -9.3072,  ...,  -1.2421,   7.0542,  -1.1888],
        ...,
        [  3.8564,  -2.1328, -14.6332,  ..., -11.3974,   0.6652,   0.5869],
        [  3.0524,  -4.3108,  12.4182,  ...,  -1.2712, -10.2992,  -1.6724],
        [  5.8812,   1.9993,  -4.7878,  ...,  -5.2116,  -1.1740,   2.2895]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [15]:

predicted, acc =  model.predict(test["mfcc"], labels, device)

print(predicted, acc)


tensor([7, 4, 4, 0, 2, 5, 7, 2, 7, 6, 2, 4, 3, 0, 3, 1, 6, 1, 3, 6, 1, 4, 4, 3,
        3, 5, 2, 7, 7, 2, 3, 6, 6, 7, 1, 6, 4, 0, 1, 7, 1, 3, 2, 2, 4, 7, 7, 3,
        0, 3, 2, 3, 7, 7, 7, 3, 3, 7, 7, 0, 0, 7, 7, 6, 5, 3, 2, 5, 6, 7, 3, 1,
        4, 5, 4, 4, 6, 7, 7, 4, 4, 4, 6, 0, 4, 2, 4, 6, 4, 7, 6, 5, 3, 4, 0, 0,
        7, 0, 4, 2, 7, 3, 6, 4, 3, 4, 0, 0, 1, 4, 3, 1, 3, 4, 5, 0, 3, 7, 0, 7,
        5, 6, 3, 2, 4, 1, 1, 4, 2, 1, 3, 0, 5, 4, 4, 7, 6, 5, 3, 4, 6, 3, 2, 7],
       device='cuda:0') 0.6041666666666666
